In [30]:
import pandas as pd
import numpy as np
import datetime
import os 

In [31]:
#read in the latest dataset
time_series = pd.read_csv("../../../databases/JHU/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,149,151,151,151,151,152,152,152,154,154
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,15,15,16,16,19,19,19,19,19,21
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,2031,2156,2173,2198,2256,2299,2329,2427,2542,2589
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,66,68,68,69,69,69,69,69,69,69


In [32]:
#keep only data from valid counties
time_series = time_series[time_series["Admin2"] != "Unassigned"]
time_series = time_series.dropna(subset = ["Admin2"])
time_series = time_series[~time_series["Admin2"].str.contains("Out of ")]
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
5,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,61,67,68,74,84,91,93,104,103,110
6,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,205,208,216,222,224,227,231,243,244,254
7,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,51,53,58,59,61,67,69,74,79,79
8,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,44,44,45,46,46,46,46,46,49,50
9,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,44,44,44,44,45,45,45,45,45,45


In [33]:
#define a dataframe with just the county information
all_counties = time_series[["FIPS", "Admin2","Province_State","Lat","Long_"]]
all_counties.head()

,FIPS,Admin2,Province_State,Lat,Long_
5,1001.0,Autauga,Alabama,32.539527,-86.644082
6,1003.0,Baldwin,Alabama,30.727750,-87.722071
7,1005.0,Barbour,Alabama,31.868263,-85.387129
8,1007.0,Bibb,Alabama,32.996421,-87.125115
9,1009.0,Blount,Alabama,33.982109,-86.567906


In [34]:
#define variable for latest day of data to be pulled
startdate = datetime.datetime(2020,3,4)
today = datetime.datetime.today()
offset = 53 + (today-startdate).days
offset

127

In [35]:
#create a dataframe that is just the number of total cases by county for each day
dates = time_series.iloc[:,53:offset]
dates.head()

,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,...,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20
5,0,0,0,0,0,0,0,0,0,0,...,61,67,68,74,84,91,93,104,103,110
6,0,0,0,0,0,0,0,0,0,0,...,205,208,216,222,224,227,231,243,244,254
7,0,0,0,0,0,0,0,0,0,0,...,51,53,58,59,61,67,69,74,79,79
8,0,0,0,0,0,0,0,0,0,0,...,44,44,45,46,46,46,46,46,49,50
9,0,0,0,0,0,0,0,0,0,0,...,44,44,44,44,45,45,45,45,45,45


In [36]:
#create an empty dataframe that will be filled with data needed to calculate new cases and 14-day rolling average
all_county_time_series = pd.DataFrame(columns = ["FIPS","Admin2","Province_State","Lat","Long_", "Confirmed",
                                             "Yesterday", "Day_3","Day_4","Day_5","Day_6","Day_7",
                                             "Day_8","Day_9","Day_10","Day_11","Day_12","Day_13","Day_14","Date"])

In [37]:
#pull the values to fill the cells for calculating new cases and 14-day rolling average
cols = dates.columns

for i in range(14,len(cols)):
    one_day = pd.DataFrame(columns = ["Confirmed", "Yesterday", "Day_3","Day_4","Day_5","Day_6","Day_7",
                                             "Day_8","Day_9","Day_10","Day_11","Day_12","Day_13","Day_14","Date"])
    one_day["Confirmed"] = dates.iloc[:,i]
    one_day["Yesterday"] = dates.iloc[:,i-1]
    one_day["Day_3"] = dates.iloc[:,i-2]
    one_day["Day_4"] = dates.iloc[:,i-3]
    one_day["Day_5"] = dates.iloc[:,i-4]
    one_day["Day_6"] = dates.iloc[:,i-5]
    one_day["Day_7"] = dates.iloc[:,i-6]
    one_day["Day_8"] = dates.iloc[:,i-7]
    one_day["Day_9"] = dates.iloc[:,i-8]
    one_day["Day_10"] = dates.iloc[:,i-9]
    one_day["Day_11"] = dates.iloc[:,i-10]
    one_day["Day_12"] = dates.iloc[:,i-11]
    one_day["Day_13"] = dates.iloc[:,i-12]
    one_day["Day_14"] = dates.iloc[:,i-13]
    one_day["Date"] = cols[i]
    all_time = all_counties.join(one_day)
    all_county_time_series = all_county_time_series.append(all_time)
#    county_time.append(one_day)
#    print(one_day)
all_county_time_series.head()


,FIPS,Admin2,Province_State,Lat,Long_,Confirmed,Yesterday,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Date
5,1001.0,Autauga,Alabama,32.539527,-86.644082,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20
6,1003.0,Baldwin,Alabama,30.727750,-87.722071,1,1,1,1,0,0,0,0,0,0,0,0,0,0,3/18/20
7,1005.0,Barbour,Alabama,31.868263,-85.387129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20
8,1007.0,Bibb,Alabama,32.996421,-87.125115,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20
9,1009.0,Blount,Alabama,33.982109,-86.567906,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20


In [38]:
all_county_time_series.reset_index(inplace = True)
all_county_time_series.drop("index", axis = 1, inplace = True)

In [39]:
all_county_time_series["Date"] = pd.to_datetime(all_county_time_series.Date)
all_county_time_series.dtypes

FIPS                     float64
Admin2                    object
Province_State            object
Lat                      float64
Long_                    float64
Confirmed                 object
Yesterday                 object
Day_3                     object
Day_4                     object
Day_5                     object
Day_6                     object
Day_7                     object
Day_8                     object
Day_9                     object
Day_10                    object
Day_11                    object
Day_12                    object
Day_13                    object
Day_14                    object
Date              datetime64[ns]
dtype: object

In [40]:
all_county_time_series.tail()

,FIPS,Admin2,Province_State,Lat,Long_,Confirmed,Yesterday,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Date
189115,NaN,Central Utah,Utah,39.372319,-111.575868,29,29,29,29,28,28,28,27,27,27,26,25,25,24,2020-05-16
189116,NaN,Southeast Utah,Utah,38.996171,-110.701396,14,13,13,13,13,14,13,13,13,12,12,12,11,11,2020-05-16
189117,NaN,Southwest Utah,Utah,37.854472,-111.441876,209,200,187,180,167,170,164,155,138,132,128,121,119,111,2020-05-16
189118,NaN,TriCounty,Utah,40.124915,-109.517442,19,16,16,16,16,16,16,16,14,14,14,14,14,14,2020-05-16
189119,NaN,Weber-Morgan,Utah,41.271160,-111.914512,221,216,214,211,207,197,194,188,184,173,173,173,167,167,2020-05-16


In [41]:
#create and calculate a column for the number of new cases each day
all_county_time_series["New_Cases"] = all_county_time_series.apply(lambda row: row.Confirmed - row.Yesterday, axis=1)

In [42]:
#create and calculate a column with a rolling average
all_county_time_series["Rolling_Avg"] = all_county_time_series.apply(
    lambda row: (row.Confirmed + row.Yesterday + row.Day_3+row.Day_4
                 +row.Day_5+row.Day_6+row.Day_7+row.Day_8+row.Day_9+
                 row.Day_10+row.Day_11+row.Day_12+row.Day_13+row.Day_14)/14, axis = 1)

In [43]:
#now that Rolling_Avg is calculated, get rid of the unneeded calculation columns
all_county_time_series = all_county_time_series[["FIPS", "Admin2", "Province_State", "Confirmed", 
                                         "New_Cases", "Rolling_Avg", "Date"]]
all_county_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
189115,NaN,Central Utah,Utah,29,0,27.214286,2020-05-16
189116,NaN,Southeast Utah,Utah,14,1,12.642857,2020-05-16
189117,NaN,Southwest Utah,Utah,209,9,155.785714,2020-05-16
189118,NaN,TriCounty,Utah,19,3,15.357143,2020-05-16
189119,NaN,Weber-Morgan,Utah,221,5,191.785714,2020-05-16


In [44]:
all_county_time_series.sort_values(["Province_State", "Admin2", "Date"], inplace = True)
all_county_time_series.reset_index(inplace = True)
all_county_time_series.drop("index", axis = 1, inplace = True)
all_county_time_series.head(5)

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
0,1001.0,Autauga,Alabama,0,0,0.0,2020-03-18
1,1001.0,Autauga,Alabama,0,0,0.0,2020-03-19
2,1001.0,Autauga,Alabama,0,0,0.0,2020-03-20
3,1001.0,Autauga,Alabama,0,0,0.0,2020-03-21
4,1001.0,Autauga,Alabama,0,0,0.0,2020-03-22


In [45]:
all_county_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
189115,56045.0,Weston,Wyoming,0,0,0.0,2020-05-12
189116,56045.0,Weston,Wyoming,0,0,0.0,2020-05-13
189117,56045.0,Weston,Wyoming,0,0,0.0,2020-05-14
189118,56045.0,Weston,Wyoming,0,0,0.0,2020-05-15
189119,56045.0,Weston,Wyoming,0,0,0.0,2020-05-16


In [46]:
#add a column to calculate rolling average of new cases
all_county_time_series["Temp"] = all_county_time_series.Rolling_Avg.shift(1)
all_county_time_series.head()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date,Temp
0,1001.0,Autauga,Alabama,0,0,0.0,2020-03-18,NaN
1,1001.0,Autauga,Alabama,0,0,0.0,2020-03-19,0.0
2,1001.0,Autauga,Alabama,0,0,0.0,2020-03-20,0.0
3,1001.0,Autauga,Alabama,0,0,0.0,2020-03-21,0.0
4,1001.0,Autauga,Alabama,0,0,0.0,2020-03-22,0.0


In [47]:
#create and calculate row with rolling average of new cases
all_county_time_series["New_Case_Roll_Avg"] = all_county_time_series.apply(lambda row: row.Rolling_Avg - row.Temp, axis=1)
all_county_time_series = all_county_time_series[all_county_time_series["Date"] != "2020-03-18"]
all_county_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date,Temp,New_Case_Roll_Avg
189115,56045.0,Weston,Wyoming,0,0,0.0,2020-05-12,0.0,0.0
189116,56045.0,Weston,Wyoming,0,0,0.0,2020-05-13,0.0,0.0
189117,56045.0,Weston,Wyoming,0,0,0.0,2020-05-14,0.0,0.0
189118,56045.0,Weston,Wyoming,0,0,0.0,2020-05-15,0.0,0.0
189119,56045.0,Weston,Wyoming,0,0,0.0,2020-05-16,0.0,0.0


In [48]:
#remove Temp column
all_county_time_series = all_county_time_series[["FIPS","Admin2","Province_State","Confirmed","Rolling_Avg",
                                       "New_Cases","New_Case_Roll_Avg","Date"]]
all_county_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,Rolling_Avg,New_Cases,New_Case_Roll_Avg,Date
189115,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-12
189116,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-13
189117,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-14
189118,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-15
189119,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-16


In [49]:
#add a column with a unique value that can be used to establish a relationship with other datasheets
all_county_time_series["Unique"] = all_county_time_series["Admin2"]+", "+all_county_time_series["Province_State"]+", "+all_county_time_series["Date"].astype(str)
all_county_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,Rolling_Avg,New_Cases,New_Case_Roll_Avg,Date,Unique
189115,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-12,"Weston, Wyoming, 2020-05-12"
189116,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-13,"Weston, Wyoming, 2020-05-13"
189117,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-14,"Weston, Wyoming, 2020-05-14"
189118,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-15,"Weston, Wyoming, 2020-05-15"
189119,56045.0,Weston,Wyoming,0,0.0,0,0.0,2020-05-16,"Weston, Wyoming, 2020-05-16"


In [50]:
all_county_time_series_change = all_county_time_series[["FIPS","Admin2","Province_State","Rolling_Avg","New_Case_Roll_Avg","Date","Unique"]]

In [51]:
#add a temporary column to calculate the change in the 14-day rolling average of new cases
timeframe = 14

all_county_time_series_change["Temp2"] = all_county_time_series_change.New_Case_Roll_Avg.shift(timeframe)
all_county_time_series_change.tail(timeframe)

C:\Users\becbo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Temp2
189106,56045.0,Weston,Wyoming,0.0,0.0,2020-05-03,"Weston, Wyoming, 2020-05-03",0.0
189107,56045.0,Weston,Wyoming,0.0,0.0,2020-05-04,"Weston, Wyoming, 2020-05-04",0.0
189108,56045.0,Weston,Wyoming,0.0,0.0,2020-05-05,"Weston, Wyoming, 2020-05-05",0.0
189109,56045.0,Weston,Wyoming,0.0,0.0,2020-05-06,"Weston, Wyoming, 2020-05-06",0.0
189110,56045.0,Weston,Wyoming,0.0,0.0,2020-05-07,"Weston, Wyoming, 2020-05-07",0.0
189111,56045.0,Weston,Wyoming,0.0,0.0,2020-05-08,"Weston, Wyoming, 2020-05-08",0.0
189112,56045.0,Weston,Wyoming,0.0,0.0,2020-05-09,"Weston, Wyoming, 2020-05-09",0.0
189113,56045.0,Weston,Wyoming,0.0,0.0,2020-05-10,"Weston, Wyoming, 2020-05-10",0.0
189114,56045.0,Weston,Wyoming,0.0,0.0,2020-05-11,"Weston, Wyoming, 2020-05-11",0.0
189115,56045.0,Weston,Wyoming,0.0,0.0,2020-05-12,"Weston, Wyoming, 2020-05-12",0.0


In [52]:
#create Change column showing the change in the 14 day rolling average over the past 14 days
all_county_time_series_change["Change"]= all_county_time_series_change.apply(lambda row: (row.New_Case_Roll_Avg - row.Temp2)/timeframe, axis = 1)
all_county_time_series_change.tail()                                                            

,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Temp2,Change
189115,56045.0,Weston,Wyoming,0.0,0.0,2020-05-12,"Weston, Wyoming, 2020-05-12",0.0,0.0
189116,56045.0,Weston,Wyoming,0.0,0.0,2020-05-13,"Weston, Wyoming, 2020-05-13",0.0,0.0
189117,56045.0,Weston,Wyoming,0.0,0.0,2020-05-14,"Weston, Wyoming, 2020-05-14",0.0,0.0
189118,56045.0,Weston,Wyoming,0.0,0.0,2020-05-15,"Weston, Wyoming, 2020-05-15",0.0,0.0
189119,56045.0,Weston,Wyoming,0.0,0.0,2020-05-16,"Weston, Wyoming, 2020-05-16",0.0,0.0


In [53]:
offset2 = datetime.timedelta(days = 2)
olddate = today - offset2
olddate

datetime.datetime(2020, 5, 15, 16, 42, 0, 911324)

In [54]:
all_county_time_series_change = all_county_time_series_change[all_county_time_series_change["Date"] > olddate]
all_county_time_series_change = all_county_time_series_change[["FIPS","Admin2","Province_State",
                                               "Rolling_Avg","New_Case_Roll_Avg","Date","Unique","Change"]]
all_county_time_series_change.tail()

,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Change
188879,56037.0,Sweetwater,Wyoming,19.642857,0.357143,2020-05-16,"Sweetwater, Wyoming, 2020-05-16",-1.530612e-02
188939,56039.0,Teton,Wyoming,98.357143,0.214286,2020-05-16,"Teton, Wyoming, 2020-05-16",-1.632653e-01
188999,56041.0,Uinta,Wyoming,8.714286,0.142857,2020-05-16,"Uinta, Wyoming, 2020-05-16",-6.344132e-17
189059,56043.0,Washakie,Wyoming,8.142857,0.071429,2020-05-16,"Washakie, Wyoming, 2020-05-16",-1.020408e-02
189119,56045.0,Weston,Wyoming,0.000000,0.000000,2020-05-16,"Weston, Wyoming, 2020-05-16",0.000000e+00


In [55]:
all_county_time_series_change.reset_index(inplace = True)
all_county_time_series_change.drop("index", axis = 1, inplace = True)
all_county_time_series_change.head(5)

,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Change
0,1001.0,Autauga,Alabama,76.214286,4.642857,2020-05-16,"Autauga, Alabama, 2020-05-16",0.229592
1,1003.0,Baldwin,Alabama,216.714286,5.214286,2020-05-16,"Baldwin, Alabama, 2020-05-16",0.005102
2,1005.0,Barbour,Alabama,59.571429,2.571429,2020-05-16,"Barbour, Alabama, 2020-05-16",0.056122
3,1007.0,Bibb,Alabama,45.214286,0.571429,2020-05-16,"Bibb, Alabama, 2020-05-16",-0.040816
4,1009.0,Blount,Alabama,43.428571,0.357143,2020-05-16,"Blount, Alabama, 2020-05-16",-0.076531


In [58]:
all_county_time_series_change["Normalized_Change"] = all_county_time_series_change.apply(lambda row: row.Change/row.New_Case_Roll_Avg if row.New_Case_Roll_Avg != 0 else 0, axis = 1)
all_county_time_series_change.head()

,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Change,Normalized_Change
0,1001.0,Autauga,Alabama,76.214286,4.642857,2020-05-16,"Autauga, Alabama, 2020-05-16",0.229592,0.049451
1,1003.0,Baldwin,Alabama,216.714286,5.214286,2020-05-16,"Baldwin, Alabama, 2020-05-16",0.005102,0.000978
2,1005.0,Barbour,Alabama,59.571429,2.571429,2020-05-16,"Barbour, Alabama, 2020-05-16",0.056122,0.021825
3,1007.0,Bibb,Alabama,45.214286,0.571429,2020-05-16,"Bibb, Alabama, 2020-05-16",-0.040816,-0.071429
4,1009.0,Blount,Alabama,43.428571,0.357143,2020-05-16,"Blount, Alabama, 2020-05-16",-0.076531,-0.214286


In [59]:
all_county_time_series["Date"] = all_county_time_series["Date"].dt.to_period("D")
all_county_time_series_change["Date"] = all_county_time_series_change["Date"].dt.to_period("D")

In [60]:
all_county_time_series.to_excel("../clean_data/all_county_rolling_avg.xlsx")
all_county_time_series_change.to_excel("../clean_data/all_county_rolling_avg_change.xlsx")